In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [9]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Regular_Array'

# MakeModule Parameters
module_type=simulationname
numpanels = 2
x = 2  # 2-up landscape
y = 1
xgap = 0.5
ygap = 0.5
zgap = 0.1
sensorsy = 9 


# TorqueTube Parameters
axisofrotationTorqueTube=True
torqueTube = True
tubetype = 'square'
diameter = 0.1
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=10
numcellsy=6

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 1  

azimuth_ang=120
tilt =30

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Noon, June 17th

# Making module with all the variables
moduledict=demo.makeModule(name=simulationname,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': 3,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname}  


scene = demo.makeScene(moduletype=module_type, sceneDict=sceneDict, nMods = 3, nRows = 2, sensorsy = sensorsy, axisofrotationTorqueTube=axisofrotationTorqueTube, diameter=diameter, zgap = zgap) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'Regular_Array')
REWRITING pre-existing module file. 
Module Regular_Array successfully created
('Axis Azimuth', None)
('Tilt', 30)
Working with offset

0.2
-0.08660254037844385 0.05000000000000001 0.17320508075688776
-1.0241025403784438 0.591265877365274 0.5482050807568879
0.1875 -0.10825317547305485 0.12499999999999999
120 30
Created Regular_Array.oct


In [ ]:
import numpy as np
dtor = np.pi/180
tilt=-45
offz = abs(0.2*np.sin(tilt*dtor))
pz = 1.25*np.sin(tilt*dtor)
print offz, "offz", pz," panel z"
zinc = 2.5/(sensorsy + 1) * np.sin(tilt*dtor)
print zinc
print 1 - pz + offz + zinc

In [ ]:
name='Mycube'
text='! genbox black cuteBox 0.05 0.05 2.5 | xform -t -0.025 -0.025 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfile, customObject, '!xform -rz 0')

name='Mycube2'
text='! genbox black cuteBox 0.1 0.1 2.5 | xform -t -3.05 -0.05 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfile, customObject, '!xform -rz 0')
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.

# At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:

rvu -vf views\front.vp -e .01 Regular_Array.oct

And then proceed happily with your analysis:

In [ ]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, "test3", scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )

In [ ]:
pitch=2.5
pitch*round(4 / 2.0)*1.0
round(4/2.0)

height = 0
sceney = 2
import numpy as np
tilt=45
dtor=np.pi/180
height + sceney *np.sin(tilt*dtor)
sceney * np.sin(tilt*dtor) / (sensorsy + 1)

azimuth = 90
sceney * np.sin((azimuth-180)*dtor) / (sensorsy + 1) * np.cos(tilt*dtor)

In [ ]:
offset = diameter + zgap

offsetx = -offset*np.sin(tilt*dtor)

offsety = -offset*np.sin(tilt*dtor)*np.cos(tilt*dtor)
azimuth = 90
tilt = 45

print abs(tilt)/tilt * offset * np.sin((azimuth-180)*dtor) / np.cos(tilt*dtor)
print offset * np.cos((azimuth-180)*dtor) / np.cos(tilt*dtor)
offsetz = offset * np.sin((azimuth-180)*dtor) / np.sin(tilt*dtor)
print offsetz, " OffsetZ"
print offsetz + zstart +zinc 

In [ ]:
scene.frontscan

In [ ]:
scene.backscan


In [ ]:
import numpy as np
y=2.5
dtor=np.pi/180
height=1
xstart = (y/2.0) * np.sin((90-tilt)*dtor) * np.cos((azimuth+90)*dtor)
ystart = (y/2.0) * np.sin((90-tilt)*dtor) * np.sin((azimuth-90)*dtor)
zstart = height + (y/2.0) * np.cos((90+tilt)*dtor) - 0.03 # + offset*np.cos(tilt*dtor)

xinc = (y/(sensorsy + 1)) * np.sin((90-tilt)*dtor) * np.cos((azimuth-90)*dtor)
yinc = (y/(sensorsy + 1)) * np.sin((90-tilt)*dtor) * np.sin((azimuth+90)*dtor)
zinc = (y/(sensorsy + 1)) * np.cos((90-tilt)*dtor)
print tilt, azimuth
print xstart+xinc, ystart+yinc, zstart+zinc
print xinc, yinc, zinc


In [ ]:
y / (9 + 1.0)

tilt


In [ ]:
tilt=-45
abs(tilt)/tilt

In [ ]:
np.sin(-45*dtor)

In [ ]:
abs(tilt)/tilt * offset * np.sin((azimuth-180)*dtor) / np.cos(tilt*dtor)
azimuth=180
np.cos((azimuth-180)*dtor)